In [ ]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.common.by import By
import re
from selenium.webdriver.firefox.options import Options

In [ ]:
url='http://ido.khjh.hc.edu.tw/moodle/'

options = Options()
options.headless = True
driverPath = '/media/shawn-pc/3T/code/python_moodlePassGrade修改及格分數/geckodriver'
driver = webdriver.Firefox(options=options, executable_path=driverPath)
#driver = webdriver.Firefox(executable_path=driverPath)

#headless模式
#options = webdriver.ChromeOptions()
#options.add_argument('--headless')
#driver = webdriver.Chrome(options=options, executable_path='chromedriver.exe')
#driver = webdriver.Chrome(executable_path='chromedriver.exe')

driver.set_page_load_timeout(60)
driver.get(url)


In [ ]:
#登入moodle
username = 'username'
password = 'password'
element = driver.find_element_by_id('login_username')
element.send_keys(username)
element = driver.find_element_by_id('login_password')
element.send_keys(password)

driver.find_element_by_css_selector(".btn-primary").click()


In [ ]:
#進入課程網址

In [ ]:
#取得測驗的連結
id = 73
courseurl = 'http://ido.khjh.hc.edu.tw/moodle/course/view.php?'
quizname=[]
quizurl=[]
for i in range(5):
    url = courseurl + 'id='+str(id) +'&section='+str(i+1)
    driver.get(url)
    #取得各section原始碼
    htmltext = driver.page_source

    soup=BeautifulSoup(htmltext,"html.parser")
    
    #搜尋所有含quiz的連結
    links = soup.find_all('a',{'href':re.compile('quiz')})
    for link in links:

        #取得href屬性前先檢查是否存在
        if 'href' in link.attrs:
            href = link['href']
            print(link.text)
            print(href)
            quizname.append(link.text)
            quizurl.append(href)
        else:
            href = None

In [ ]:
#從url找id
quizid=[]
for quiz in quizurl:
    k = re.search(r'id=(.*)$', quiz)
    quizid.append(k.group(1))
quizid

In [ ]:
#進入試卷編修網址
quizurlBase = 'http://ido.khjh.hc.edu.tw/moodle/course/modedit.php?update='
n = 0
gradepass = 100  #及格分數
attempts = 5      #可測驗次數
for i in quizid:
    quizurl = quizurlBase+str(i)+'&return=1'
    
    driver.get(quizurl)

    #展開全部
    driver.find_element_by_css_selector('.collapseexpand').click()

    #修改及格分數
    element =  driver.find_element_by_id('id_gradepass')
    element.clear()
    element.send_keys(gradepass)

    #修改允許作答次數
    #搜尋作答次數的選單並點擊
    element = driver.find_element_by_id('id_attempts')
    element.click()
    element.send_keys(attempts)

    
    
    #解開完成進度的選項
    driver.find_element_by_id('id_unlockcompletion').click()


    #儲存並顯示
    driver.find_element_by_id('id_submitbutton2').click()
    print('done:' + quizname[n])
    print(quizurl)
    n = n+1
    print(str(n)+"/"+str(len(quizid)))
    print('========')


## 以下是測試區

In [ ]:
#處理單一連結的測試區

#進入試卷編修網址
url = 'http://ido.khjh.hc.edu.tw/moodle/course/modedit.php?update=5827&return=1'
driver.get(url)

#element = WebDriverWait(driver,3).until(expected_conditions.presence_of_element_located((By.ID,'id_gradepass')))
#element = driver.find_element_by_name('gradepass')

#展開全部
driver.find_element_by_css_selector('.collapseexpand').click()

#修改及格分數
element =  driver.find_element_by_id('id_gradepass')
element.clear()
element.send_keys('100')

#修改允許作答次數
#搜尋作答次數的選單並點擊
element = driver.find_element_by_id('id_attempts')
element.click()
element.send_keys('7')

#for option in driver.find_elements_by_tag_name('option'):
    #if option.text == '3':
    #    option.click()



#解開完成進度的選項
driver.find_element_by_id('id_unlockcompletion').click()


#儲存並顯示
driver.find_element_by_id('id_submitbutton2').click()
print('done')
#element = driver.find_element_by_xpath('//*[@id="id_gradepass"]')


#finally:
    #driver.quit()
